In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.client import device_lib
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
import requests

df = pd.read_csv("/content/drive/MyDrive/навчальне/data/netflix_titles.csv")

df

In [ ]:
df = df.dropna()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df = df.drop(["description", "duration", "listed_in", "cast", "date_added", "title", "director"], axis = 1)

In [ ]:
df

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

df['show_id'] = encoder.fit_transform(df['show_id'])
df['type'] = encoder.fit_transform(df['type'])
df['country'] = encoder.fit_transform(df['country'])
df['rating'] = encoder.fit_transform(df['rating'])

In [ ]:
df

In [ ]:
print('country:', df['country'].nunique(), '\n')
print('release_year:', df['release_year'].nunique(), '\n')
print('rating:', df['rating'].nunique(), '\n')

In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [ ]:
X = df.drop(["rating"], axis=1)
y = df["rating"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 42)

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

KERAS

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU

model = keras.Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='softmax'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='relu'))
model.add(Dense(1, activation='softmax'))

model.summary()
from keras.optimizers import Adam
from keras.optimizers import SGD
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

In [ ]:
loss, accuracy = model.evaluate(X_train, y_train)
print(f'accuracy: {accuracy*100}%')

IMAGE

In [ ]:

import matplotlib.pyplot as plt
import gdown
from os import listdir
from sklearn.preprocessing import LabelBinarizer
import random
from matplotlib.image import imread
from PIL import Image
from keras.preprocessing.image import img_to_array, array_to_img
from keras.preprocessing import image
from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense, LeakyReLU

import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as image

import itertools
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.metrics import classification_report, f1_score , confusion_matrix


image_df =  pd.read_csv("/content/drive/MyDrive/навчальне/data/lab3/sports.csv")

In [ ]:
image_df

In [ ]:
print(f"Total number of target classes : {len(image_df.labels.unique())}")

In [ ]:
labels_to_keep = ['disc golf', 'field hockey', 'golf', 'hurdles', 'polo']

image_df = image_df[image_df['labels'].isin(labels_to_keep)]

image_df

In [ ]:
print(f"Total number of target classes : {len(image_df.labels.unique())}")

In [ ]:
dataset = {
             "train_data" : "/content/drive/MyDrive/навчальне/data/lab3/train",
             "valid_data" : "/content/drive/MyDrive/навчальне/data/lab3/valid",
             "test_data" : "/content/drive/MyDrive/навчальне/data/lab3/test"
          }

all_data = []
for path in dataset.values():
    data = {"imgpath": [] , "labels": [] }
    category = os.listdir(path)

    for folder in category:
        folderpath = os.path.join(path , folder)
        filelist = os.listdir(folderpath)
        for file in filelist:
            fpath = os.path.join(folderpath, file)
            data["imgpath"].append(fpath)
            data["labels"].append(folder)


    all_data.append(data.copy())
    data.clear()



train_df = pd.DataFrame(all_data[0] , index=range(len(all_data[0]['imgpath'])))
valid_df = pd.DataFrame(all_data[1] , index=range(len(all_data[1]['imgpath'])))
test_df = pd.DataFrame(all_data[2] , index=range(len(all_data[2]['imgpath'])))


# #Convert labels to numbers
lb = LabelEncoder()
train_df['encoded_labels'] = lb.fit_transform(train_df['labels'])
valid_df['encoded_labels'] = lb.fit_transform(valid_df['labels'])
test_df['encoded_labels'] = lb.fit_transform(test_df['labels'])

In [ ]:
valid_df = valid_df[valid_df['labels'].isin(labels_to_keep)]
train_df = train_df[train_df['labels'].isin(labels_to_keep)]
test_df = test_df[test_df['labels'].isin(labels_to_keep)]

In [ ]:
plt.figure(figsize=(15,12))
for i, row in valid_df.sample(n=25).reset_index().iterrows():
    plt.subplot(5,5,i+1)
    image_path = row['imgpath']
    image = Image.open(image_path)
    plt.imshow(image)
    plt.title(row["labels"])
    plt.axis('off')
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Dropout , BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers,models,Model
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import mixed_precision



BATCH_SIZE = 10
IMAGE_SIZE = (224, 224)


generator = ImageDataGenerator(
    preprocessing_function = tf.keras.applications.efficientnet.preprocess_input,
    # there could be image augmentation here
)

# Split the data into three categories.
train_images = generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='imgpath',
    y_col='labels',
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
)

valid_images = generator.flow_from_dataframe(
    dataframe=valid_df,
    x_col='imgpath',
    y_col='labels',
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

test_images = generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='imgpath',
    y_col='labels',
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [ ]:
pretrained_model = tf.keras.applications.EfficientNetB0(
    input_shape=(224, 224, 3),
    include_top=False, # we don`t need a pre-trained top layer (output layer)
    weights='imagenet',
    pooling='max'
)

# Freezing the layers of a pretrained neural network
for i, layer in enumerate(pretrained_model.layers):
    pretrained_model.layers[i].trainable = False

In [ ]:
num_classes = len(set(train_images.classes))


# Data Augmentation Step
augment = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal"),
  layers.experimental.preprocessing.RandomRotation(0.1),
  layers.experimental.preprocessing.RandomZoom(0.1),
  layers.experimental.preprocessing.RandomContrast(0.1),
], name='AugmentationLayer')



inputs = layers.Input(shape = (224,224,3), name='inputLayer')
x = augment(inputs)
pretrain_out = pretrained_model(x, training = False)
x = layers.Dense(350)(pretrain_out)
x = layers.Activation(activation="relu")(x)
x = BatchNormalization()(x)
x = layers.Dropout(0.25)(x)
x = layers.Dense(num_classes)(x)
outputs = layers.Activation(activation="softmax", dtype=tf.float32, name='activationLayer')(x) # mixed_precision need separated Dense and Activation layers
model = Model(inputs=inputs, outputs=outputs)



model.compile(
    optimizer=Adam(0.0005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(model.summary())

In [ ]:
history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=valid_images,
    validation_steps=len(valid_images),
    epochs=50,
    callbacks=[
        EarlyStopping(monitor = "val_loss",
                               patience = 3,
                               restore_best_weights = True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, mode='min')
    ]
)
model.save_weights('/content/drive/MyDrive/навчальне/data')

In [ ]:
results = model.evaluate(test_images, verbose=0)

print("Loss: {:.5f}".format(results[0]))
print("Accuracy: {:.2f}%".format(results[1] * 100))

TEXT

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import imdb

In [ ]:
df_text = pd.read_csv("/content/drive/MyDrive/навчальне/data/lab3.2/bible_data_set.csv")

df_text

In [ ]:
df_text = df_text[['book', 'text']]

df_text

In [ ]:
X_text3 = df_text['text']
Y_text3 = df_text['book']


from sklearn.model_selection import train_test_split
X_text3_train, X_text3_test, Y_text3_train, Y_text3_test = train_test_split(X_text3, Y_text3,  test_size=0.20)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_text3_train)
X_text3_train_seq = tokenizer.texts_to_sequences(X_text3_train)
X_text3_test_seq = tokenizer.texts_to_sequences(X_text3_test)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Padding
max_sequence_length = 100
X_train_pad = pad_sequences(X_text3_train_seq, maxlen=max_sequence_length)
X_test_pad = pad_sequences(X_text3_test_seq, maxlen=max_sequence_length)

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping

model3 = keras.Sequential()
model3.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_sequence_length))
model3.add(LSTM(100))
model3.add(Dense(3, activation='sigmoid'))
model3.add(Dense(3, activation='softmax'))

In [ ]:
model3.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model3.summary()

In [ ]:
model3.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = 10
batch_size = 64
history = model3.fit(X_train_pad, Y_text3_train, epochs=epochs, batch_size=batch_size)

In [ ]:
loss, accuracy = model3.evaluate(X_test_pad, Y_text3_test)
print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')